# alphavec example

This notebook demonstrates running `alphavec.simulate()` on the bundled feather datasets in `tests/testdata/`.

Notes:
- The feather files require `pyarrow`. If you don't have it, install with `pip install pyarrow` in your venv.
- NaNs in prices or funding indicate an asset is not tradable at that period; `simulate()` respects this.
- The run below uses all assets over the full available history.


In [ ]:
from pathlib import Path

import pandas as pd

from alphavec import simulate


In [ ]:
from IPython.display import HTML, display
import html as _html


def _format_metric_value(value) -> str:
    if value is None:
        return ""
    if isinstance(value, float) and pd.isna(value):
        return ""
    if isinstance(value, pd.Timestamp):
        return value.isoformat()
    if isinstance(value, int):
        return f"{value:,}"
    if isinstance(value, float):
        return f"{value:,.6f}"
    return str(value)


def render_tearsheet(tearsheet: pd.DataFrame, *, caption: str = "Tearsheet") -> HTML:
    df = tearsheet.copy()
    df["Value"] = df["Value"].map(_format_metric_value)

    def _format_note(value) -> str:
        if value is None:
            return ""
        if isinstance(value, float) and pd.isna(value):
            return ""
        return str(value)

    caption_html = f"<caption>{_html.escape(caption)}</caption>" if caption else ""
    rows: list[str] = []
    for metric, row in df.iterrows():
        metric_html = _html.escape(str(metric))
        value_html = _html.escape(str(row.get("Value", "")))
        note_html = _html.escape(_format_note(row.get("Note", "")))
        rows.append(
            "<tr>"
            f"<th scope=\"row\">{metric_html}</th>"
            f"<td class=\"av-ts-value\">{value_html}</td>"
            f"<td class=\"av-ts-note\">{note_html}</td>"
            "</tr>"
        )

    table_html = (
        '<table class="alphavec-tearsheet">'
        + caption_html
        + "<thead><tr><th>Metric</th><th>Value</th><th>Note</th></tr></thead>"
        + "<tbody>"
        + "".join(rows)
        + "</tbody></table>"
    )

    return HTML(table_html)


In [ ]:
data_dir = Path(".")

weights = pd.read_feather(data_dir / "weights.feather")
close_prices = pd.read_feather(data_dir / "close_prices.feather")
order_prices = pd.read_feather(data_dir / "open_prices.feather")
funding_rates = pd.read_feather(data_dir / "funding_rates.feather")

print("weights", weights.shape)
print("close_prices", close_prices.shape)
print("order_prices", order_prices.shape)
print("funding_rates", funding_rates.shape)


In [ ]:
symbols = weights.columns.tolist()
print("assets", len(symbols))
print("start", weights.index.min())
print("end", weights.index.max())

na_summary = {
    "weights_na": int(weights.isna().sum().sum()),
    "close_na": int(close_prices.isna().sum().sum()),
    "order_na": int(order_prices.isna().sum().sum()),
    "funding_na": int(funding_rates.isna().sum().sum()),
}
na_summary


In [ ]:
# Use all assets and the full time range.
weights_all = weights.sort_index()

# Drop leading periods before the strategy has any active (non-NaN) weights.
active_mask = weights_all.notna().any(axis=1)
if not bool(active_mask.any()):
    raise ValueError("weights contains no non-NaN rows.")
weights_all = weights_all.loc[active_mask.idxmax() :]

# Align prices/funding to weights; NaNs mean not tradable.
close_all = close_prices.reindex(index=weights_all.index, columns=weights_all.columns)
# Use next-period open as the execution price to avoid lookahead.
open_all = order_prices.reindex(index=weights_all.index, columns=weights_all.columns)
order_all = open_all.shift(-1)
funding_all = funding_rates.reindex(index=weights_all.index, columns=weights_all.columns)

weights_all.head()


In [ ]:
init_cash = 10_000
benchmark_asset = "BTC"  # set to None to disable alpha/beta

returns, tearsheet = simulate(
    weights=weights_all,
    close_prices=close_all,
    order_prices=order_all,
    funding_rates=funding_all,
    benchmark_asset=benchmark_asset,
    order_notional_min=10,
    fee_pct=0.00025,       # 2.5 bps per trade
    slippage_pct=0.001,  # 10 bps per trade
    init_cash=init_cash,
    freq_rule="1D",
    trading_days_year=365,
    risk_free_rate=0.03,
)

display(returns.head())
render_tearsheet(tearsheet)


In [ ]:
equity = (1.0 + returns).cumprod() * init_cash

benchmark_equity = None
if benchmark_asset is not None:
    bench_prices = close_all[benchmark_asset].copy().ffill().bfill()
    bench_returns = bench_prices.pct_change().fillna(0.0)
    benchmark_equity = (1.0 + bench_returns).cumprod() * init_cash

equity.tail()


## Interactive charts (Plotly)

These charts visualize the portfolio equity, drawdowns, and return distribution.


In [ ]:
try:
    import plotly.express as px
    import plotly.graph_objects as go
except ModuleNotFoundError as e:
    raise ModuleNotFoundError(
        "plotly is required for these charts. Install in your venv: pip install plotly"
    ) from e


In [ ]:
# Portfolio + benchmark equity curve
equity_plot = equity.rename("Portfolio").to_frame().rename_axis("ts")
if benchmark_equity is not None:
    equity_plot[f"Benchmark ({benchmark_asset})"] = benchmark_equity

equity_long = equity_plot.reset_index().melt(
    id_vars="ts",
    var_name="series",
    value_name="equity",
)

fig_equity = px.line(
    equity_long,
    x="ts",
    y="equity",
    color="series",
    title="Equity Curve",
)
fig_equity.update_layout(yaxis_title="Equity", legend_title_text="")
fig_equity.show()


In [ ]:
# Drawdown series
drawdown = equity / equity.cummax() - 1.0
dd_df = drawdown.reset_index()
dd_df.columns = ["ts", "drawdown"]

fig_dd = px.area(
    dd_df,
    x="ts",
    y="drawdown",
    title="Portfolio Drawdown",
)
fig_dd.update_layout(yaxis_title="Drawdown")
fig_dd.update_yaxes(tickformat=".1%")
fig_dd.add_hline(y=0, line_width=1, line_dash="dash", line_color="gray")
fig_dd.show()


In [ ]:
# Returns distribution
ret_df = returns.to_frame("returns")

fig_hist = px.histogram(
    ret_df,
    x="returns",
    nbins=100,
    title="Returns Distribution",
    marginal="box",
)
fig_hist.update_layout(xaxis_title="Period Return", yaxis_title="Count")
fig_hist.update_xaxes(tickformat=".2%")
fig_hist.show()
